In [1]:
import os

def total_files(folder_path):
    num_files = len(f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)))
    return num_files

train_files_healthy = "Dataset/Train/Train/Healthy"
train_files_powdery = "Dataset/Train/Train/Powdery"
train_files_rust = "Dataset/Train/Train/Rust"

test_files_healthy = "Dataset/Test/Test/Healthy"
test_files_powdery = "Dataset/Test/Test/Powdery"
test_files_rust = "Dataset/Test/Test/Rust"

In [4]:
from keras.preprocessing.image import ImageDataGenerator as idg

In [6]:
train_datagen = idg(rescale = 1./255, shear_range = 0.2, zoom_range= 0.2, horizontal_flip = True)
test_datagen = idg(rescale = 1./255)

In [7]:
train_generator = train_datagen.flow_from_directory("Dataset/Train/Train", target_size=(225,225), batch_size=32, class_mode="categorical")

validation_generator = test_datagen.flow_from_directory("Dataset/Validation/Validation", target_size=(225, 225), batch_size = 32, class_mode = "categorical")

Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten , Dense


model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (225, 225, 3) , activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ["accuracy"])

In [10]:
history = model.fit(train_generator, batch_size=16, epochs = 5, validation_data=validation_generator, validation_batch_size=16)

Epoch 1/5
42/42 [==============================] - 185s 4s/step - loss: 1.9311 - accuracy: 0.4667 - val_loss: 0.7305 - val_accuracy: 0.5667
Epoch 2/5
42/42 [==============================] - 191s 5s/step - loss: 0.8041 - accuracy: 0.5923 - val_loss: 0.7139 - val_accuracy: 0.6500
Epoch 3/5
42/42 [==============================] - 246s 6s/step - loss: 0.6392 - accuracy: 0.7027 - val_loss: 0.6656 - val_accuracy: 0.6833
Epoch 4/5
42/42 [==============================] - 250s 6s/step - loss: 0.4731 - accuracy: 0.7965 - val_loss: 0.5232 - val_accuracy: 0.7833
Epoch 5/5
42/42 [==============================] - 244s 6s/step - loss: 0.4050 - accuracy: 0.8366 - val_loss: 0.5205 - val_accuracy: 0.7500


In [11]:
history = model.fit(train_generator, batch_size=16, epochs = 2, validation_data=validation_generator, validation_batch_size=16)

Epoch 1/2
42/42 [==============================] - 246s 6s/step - loss: 0.3314 - accuracy: 0.8691 - val_loss: 0.5408 - val_accuracy: 0.8500
Epoch 2/2
42/42 [==============================] - 239s 6s/step - loss: 0.2591 - accuracy: 0.9153 - val_loss: 0.4890 - val_accuracy: 0.8333


In [13]:
model.save("model.h5")

In [15]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def preprocess(image_path, target_size = (225, 225)):
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)
    x = x.astype("float32")/ 255
    x = np.expand_dims(x, axis = 0)
    return x

x = preprocess("Dataset/Test/Test/Rust/82f49a4a7b9585f1.jpg")

In [16]:
prediction = model.predict(x)
prediction[0]

array([9.9562123e-02, 4.5072194e-04, 8.9998710e-01], dtype=float32)

In [17]:
labels = train_generator.class_indices
labels = {v: k for k, v in labels.items()}
labels

{0: 'Healthy', 1: 'Powdery', 2: 'Rust'}

In [19]:
pred = labels[np.argmax(prediction)]
print(pred)

Rust
